# Subbundles Part 3: Streamline Profiles

**Subbundle** - a subgroup of streamlines with a set of common properties

Part 3: Get streamline profiles for tissue properties `fa_values` and `md_values`

In [ ]:
from utils import *

import time
import os.path as op

import numpy as np
import pandas as pd

from dipy.io.streamline import load_tractogram
from dipy.stats.analysis import afq_profile, gaussian_weights
from dipy.tracking.streamline import set_number_of_points, values_from_volume

import nibabel as nib

from AFQ import api
import AFQ.data as afd

from fastdtw import fastdtw

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Streamlines (from Part 2)

#### AFQ

- Instantiate AFQ object: `myafq` for desired dataset

- get `row` from `myafq` to interact with api

In [ ]:
compare_test_retest = False

test_retest_dir = 'HCP_test_retest'
test_retest_sessions = ['test', 'retest']
test_retest_names = ['HCP', 'HCP_retest']

# dataset_name = 'HCP'
dataset_name = 'HCP_retest'

In [ ]:
# subjects = get_subjects(dataset_name)
subjects = get_subjects_small(dataset_name)
# subjects = get_subjects_medium(dataset_name)

In [ ]:
if compare_test_retest:
    sub_dir = test_retest_dir
    
    print('HCP')
    myafq_test = get_afq('HCP')
    display(myafq_test.data_frame)
    
    print('HCP_retest')
    myafq_retest = get_afq('HCP_retest')
    display(myafq_retest.data_frame)
else:
    sub_dir = dataset_name
    
    print(dataset_name)
    myafq = get_afq(dataset_name)
    display(myafq.data_frame)

In [ ]:
# if compare_test_retest:
#     bundle_names = [*myafq_retest.bundle_dict]
# else:
#     bundle_names = [*myafq.bundle_dict]

# bundle_names = ['SLF_L', 'SLF_R']
# bundle_names = ['ARC_L', 'ARC_R', 'CST_L', 'CST_R', 'FP'] 
bundle_names = ['SLF_L', 'SLF_R', 'ARC_L', 'ARC_R', 'CST_L', 'CST_R', 'FP']

informational

In [ ]:
tg_fnames = {}
tractograms = {}
streamlines = {}
affines = {}

for bundle_name in bundle_names:
    if compare_test_retest:
        make_dirs(myafq_retest, sub_dir, bundle_name, subjects)
    else:
        make_dirs(myafq, sub_dir, bundle_name, subjects)
    
for subject in subjects:
    tg_fnames[subject] = {}
    tractograms[subject] = {}
    streamlines[subject] = {}
    affines[subject] = {}
    
    if compare_test_retest:
        loc_test  = get_iloc(myafq_test, subject)
        loc_retest = get_iloc(myafq_retest, subject)
    else:
        loc = get_iloc(myafq, subject)

    for bundle_name in bundle_names:
        tg_fnames[subject][bundle_name] = {}
        tractograms[subject][bundle_name] = {}
        streamlines[subject][bundle_name] = {}
        affines[subject][bundle_name] = {}

        if compare_test_retest:
            iterables = zip(test_retest_names, test_retest_sessions, [myafq_test, myafq_retest], [loc_test, loc_retest])
        else:
            iterables = zip([sub_dir], [dataset_name], [myafq], [loc])
            
        for name, ses, myafq, loc in iterables:
            tg_fnames[subject][bundle_name][ses] = get_tractogram_filename(myafq, bundle_name, loc)
            tractograms[subject][bundle_name][ses] = load_tractogram(tg_fnames[subject][bundle_name][ses], 'same')
            streamlines[subject][bundle_name][ses] = tractograms[subject][bundle_name][ses].streamlines
            affines[subject][bundle_name][ses] = tractograms[subject][bundle_name][ses].affine
            
tg_df = pd.DataFrame.from_dict(
    {(i,j,k): [len(tractograms[i][j][k].streamlines), tractograms[i][j][k].affine, tg_fnames[i][j][k]] for i in tg_fnames.keys() for j in tg_fnames[i].keys() for k in tg_fnames[i][j].keys()}, 
    orient='index', 
    columns=['number of streamlines', 'affine', 'tratogram files']
)

with pd.option_context('display.max_colwidth', -1):
    display(tg_df)

## Generate Streamline Tract Profiles

<span style="color:blue">**TODO: Quantify *Stability* and *Robustness* of resulting tract profiles**</span>

Verify that tract profiles are:

- invariant to tractometry (same dataset, subject, and session)

- similar across sessions (same dataset and subject)

- similar across subjects (same dataset)

- similar across datasets

**These should align with published tolereances.**

<span style="color:blue">**TODO: find published sources**</span>

### Streamline Tract Profile Metrics

Calculate Streamline Tract Profiles, using:

- (DTI/DKI) Tissue Properties

  - Other Tissue Properties

- (Geometrically Constrained) Distance

#### (DTI/DKI) Tissue Properties:

  - **[FA](https://en.wikipedia.org/wiki/Fractional_anisotropy) (*Fractional Anisotropy*)**
  
  - **MD (*Mean Diffusivity*)**
  
  - APM (*Anisotropic Power Map*)


#### (Geometrically Constrained) Distance:

  - Euclidean
  
  - QuickBundle MDF (*Minimum Average Direct Flip*)


##### Other Tissue Properties:

- [Relaxometry](https://radiopaedia.org/articles/relaxometry?lang=us)-based parameter

- **T<sub>1</sub>/T<sub>2</sub> ratio** (**NOTE: May not exist for HARDI dataset**)

- Quantitative T<sub>1</sub> (**NOTE: May not exist for HCP dataset**)
    
    
- quantitative measures (T<sub>1</sub>, T<sub>2</sub>, T<sub>2</sub>*)
  
- semi-quantitative measures (T<sub>2</sub>-weighted/T<sub>1</sub>-weighted ratio (T<sub>2</sub>w/T<sub>1</sub>w))


##### Cortical Endpoints:

See how close streamlines endpoints are to gray matter. **NOTE: @arokem: can determine from imaging information**

- <span style="color:red">**Question: how to use information?**<span>
    
    - as additional adjacency matricies, or
    
    - as input to topologically transform streamlines)

### Get Tissue Properties (Scalar Data)

#### Diagnostic information:

#### Get scalar data

In [ ]:
print('Scalars')

if compare_test_retest:
    print('test', myafq_test.scalars)
    print('retest', myafq_retest.scalars)
else:
    print(myafq.scalars)

scalars = ['dti_fa']

scalar_files = {}
scalar_data = {}

for subject in subjects:
    scalar_files[subject] = {}
    scalar_data[subject] = {}

    if compare_test_retest:
        loc_test  = get_iloc(myafq_test, subject)
        loc_retest = get_iloc(myafq_retest, subject)
        
        scalar_files[subject]['test'] = {}
        scalar_files[subject]['retest'] = {}
        
        scalar_data[subject]['test'] = []
        scalar_data[subject]['retest'] = []
        
        iterables = zip(test_retest_names, test_retest_sessions, [myafq_test, myafq_retest], [loc_test, loc_retest])
    else:
        loc = get_iloc(myafq, subject)
        
        scalar_files[subject][dataset_name] = {}
        
        scalar_data[subject][dataset_name] = []

        iterables = zip([sub_dir], [dataset_name], [myafq], [loc])

    for name, ses, myafq, loc in iterables:
        for scalar in scalars:    
            scalar_file = get_scalar_filename(myafq, scalar, loc)
            scalar_files[subject][ses][scalar] = scalar_file
            scalar_data[subject][ses].append(nib.load(scalar_file).get_fdata())

scalars_df = pd.DataFrame.from_dict(
    {(i,j,k): scalar_files[i][j][k] for i in scalar_files.keys() for j in scalar_files[i].keys() for k in scalar_files[i][j].keys()}, 
    orient='index', 
    columns=['scalar files']
)

with pd.option_context('display.max_colwidth', -1):
    display(scalars_df)
    
os.makedirs(op.join('subbundles', dataset_name), exist_ok=True)
f_name = op.join('subbundles', dataset_name, f'scalar_files.csv')
print(f_name)
scalars_df.to_csv(f_name)

<span style="color:blue">**TODO: `anisotropic_power_map`**</span>

## Streamline Profiles

Calculate tissue properties per streamline

For reference:
- https://github.com/dipy/dipy/blob/master/dipy/tracking/streamline.py#L668

  - https://github.com/dipy/dipy/blob/master/dipy/stats/analysis.py#L221

### Streamline Profile

##### <span style="color:red">NOTE: By default sampling `100` points from each streamline</span>

- The choice of `n_points` by convenction, however want to ensure it is greater than minimum number in `streamlines`.

In [ ]:
n_points = 100
use_weighted_means = False

Original streamline profiles. Shows that the mean is not best fit, and possiblity that streamline profiles are

- random (the grouping streamlines for bundle is arbitrary)

- offset (the bundle is a single entity, but streamlines are shifted)

- subbundles (the bundle has two or more subbundles)

or some combination thereof

In [ ]:
# TODO load from file instead of recalculating
for subject in subjects:

    if compare_test_retest:
        loc_test  = get_iloc(myafq_test, subject)
        loc_retest = get_iloc(myafq_retest, subject)
    else:
        loc = get_iloc(myafq, subject)

    for bundle_name in bundle_names:
        if compare_test_retest:
            iterables = zip(test_retest_names, test_retest_sessions, [myafq_test, myafq_retest], [loc_test, loc_retest])
        else:
            iterables = zip([sub_dir], [dataset_name], [myafq], [loc])
            
        for name, ses, myafq, loc in iterables:
            print(name, subject, bundle_name, ses)
            fgarray = set_number_of_points(streamlines[subject][bundle_name][ses], n_points)
            
            if len(fgarray) == 0:
                print(f'No streamlines for {name} {subject} {bundle_name} {ses}')
                continue

            for scalar_name, data in zip(scalars, scalar_data[subject][ses]):
                truncated_name = scalar_name.split('_')[-1]
                target_dir = get_dir_name(myafq, sub_dir, bundle_name, loc)
                
                values = np.array(values_from_volume(data, fgarray, affines[subject][bundle_name][ses]))
#                 print(values)
                print(values.shape)
#                 print(len(values)==len(streamlines[subject][bundle_name][ses]))

                if not compare_test_retest:
                    f_name = op.join(target_dir, f'streamline_profile_{truncated_name}.npy')
                    print(f_name)
                    np.save(f_name, values)

                if use_weighted_means:
                    mean_values = afq_profile(
                            data,
                            streamlines[subject][bundle_name][ses],
                            affines[subject][bundle_name][ses],
                            weights=gaussian_weights(streamlines[subject][bundle_name][ses])
                    )
                else:
                    mean_values = np.mean(values, axis=0)
                    
                if not compare_test_retest:
                    reference_name = 'mean'
                    
                    if use_weighted_means:
                        reference_name = 'weighted_mean'
                        
                    f_name = op.join(target_dir, f'streamline_profile_{reference_name}_{truncated_name}.npy')
                    print(f_name)
                    np.save(f_name, values)

                plt.figure()
                plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} profiles\n streamline count:{len(tractograms[subject][bundle_name][ses].streamlines)}')
                plt.plot(values.T, c='tab:blue', alpha=0.1)
                plt.plot(mean_values.T, c='black', label='bundle mean ($\mu$)')
                plt.xlabel('node index')
                plt.ylabel(f'{scalar_name} values')
                plt.legend()
                f_name = op.join(target_dir, f'streamline_{truncated_name}_profile.png')
                print(f_name)
                plt.savefig(f_name, bbox_inches = "tight")
                plt.show()

                fig = plt.figure()
                plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} heatmap\n streamline count:{len(tractograms[subject][bundle_name][ses].streamlines)}')
                im = plt.imshow(values.T, cmap='hot', interpolation='nearest')
                plt.xlabel('streamline index')
                plt.ylabel('node index')
                add_colorbar(im)
                f_name = op.join(target_dir, f'streamline_{truncated_name}_heatmap.png')
                print(f_name)
                plt.savefig(f_name, bbox_inches = "tight")
                plt.show()

#### Aligned Streamline Profiles

##### <span style="color:red">NOTE: Alignment Problem</span>

Streamlines do not have the same length

Streamlines do not necessarily begin and end in same regions

Streamlines may not terminate in or near gray matter

- Some of the above concerns may be addressed downstream when clustering by adding additional metrics:

  - Incorporate geometric distance
  
  - Incorporate streamline profiles for different streamline alignments

For comparison (see Diffusion profile realignment (dpr)):

https://github.com/samuelstjean/dpr

##### Dynamic Time Warping (DTW)

<span style="color:blue">**TODO: add quartiles**</span>

<span style="color:red">**NOTE: Reference: raw mean**</span>

- Alternative references includ weighted average (afq_profile), centriod (QuickBundle), ...

<span style="color:red">**NOTE: Multivaluedness: to convert to single value; chioce: grabs the last value in sequence**</span>

 - Concerns over principled approach to resolve multivaluedness and reduce to single valued mapping
 
   - Looking for deterministic, informed and reason for choices
   
 - Approach
 
   - for a given reference tissue profile: find the corresponding index to streamline profile index(es)
   
     - if multivalued: then _choose_ one of: first, last; min, max; average: mean, mode, median; wieghted average

##### $\mu$-Reference Warped Streamlines

In [ ]:
show_warp_map = True
show_baseline = False

In [ ]:
# TODO load from file instead of recalculating
for subject in subjects:
    if compare_test_retest:
        loc_test  = get_iloc(myafq_test, subject)
        loc_retest = get_iloc(myafq_retest, subject)
    else:
        loc = get_iloc(myafq, subject)
        
    for bundle_name in bundle_names:
        if compare_test_retest:
            iterables = zip(test_retest_names, test_retest_sessions, [myafq_test, myafq_retest], [loc_test, loc_retest])
        else:
            iterables = zip([sub_dir], [dataset_name], [myafq], [loc])
            
        for name, ses, myafq, loc in iterables:
            print(name, subject, bundle_name, ses)
            target_dir = get_dir_name(myafq, sub_dir, bundle_name, loc)
            fgarray = set_number_of_points(streamlines[subject][bundle_name][ses], n_points)
            
            if len(fgarray) == 0:
                print(f'No streamlines for {name} {subject} {bundle_name} {ses}')
                continue
            
            for scalar_name, data in zip(scalars, scalar_data[subject][ses]):
                truncated_name = scalar_name.split('_')[-1]
                values = np.array(values_from_volume(data, fgarray, affines[subject][bundle_name][ses]))
                
                if use_weighted_means:
                    mean_values = afq_profile(
                            data,
                            streamlines[subject][bundle_name][ses],
                            affines[subject][bundle_name][ses],
                            weights=gaussian_weights(streamlines[subject][bundle_name][ses])
                    )
                else:
                    mean_values = np.mean(values, axis=0)
                    
                if show_warp_map:
                    plt.figure()
                    plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} mean ($\mu$)-reference warping map')
                    for value in values:
                        plt.plot(*zip(*fastdtw(value, mean_values)[1]), c='tab:blue')
                    plt.xlabel('input node index')
                    plt.ylabel('output node index')
                    f_name = op.join(target_dir, f'{truncated_name}_warp_map.png')
                    print(f_name)
                    plt.savefig(f_name, bbox_inches = "tight")
                    plt.show()

                tic = time.perf_counter()
                dtw_values = []

                for value in values:
                    dist, path = fastdtw(value, mean_values)

                    path = np.array(path)

                    dtw_values.append(value[np.append(path[np.where(path[:,1][:-1] != path[:,1][1:]),0][0], len(values.T)-1)])

                dtw_values = np.array(dtw_values)
                toc = time.perf_counter()
                print(f'dtw calculation {toc - tic:0.4f} seconds')
                print(dtw_values.shape)
                
                if not compare_test_retest:
                    f_name = op.join(target_dir, f'streamline_profile_ref_warped_{truncated_name}.npy')
                    print(f_name)
                    np.save(f_name, dtw_values)

                # cacluate warped bundle profile
                if use_weighted_means:
                    # TODO
                    print('weighed warped mean not implemented!')
                    continue
                else:
                    dtw_mean_values = np.mean(dtw_values, axis=0)
                
                if not compare_test_retest:
                    reference_name = 'mean'
                    
                    if use_weighted_means:
                        reference_name = 'weighted_mean'
                        
                    f_name = op.join(target_dir, f'streamline_profile_ref_warped_{reference_name}_{truncated_name}.npy')
                    print(f_name)
                    np.save(f_name, values)

                # show original streamline profiles corresponding to the warp
                #  duplicates plots from above, but useful so don't have to scroll 
                #  or if only interested in running and saving warps
                if show_baseline:
                    plt.figure()
                    plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} streamline profiles')
                    plt.plot(values.T, c='tab:blue', alpha=0.1)
                    plt.plot(mean_values.T, c='black', label='bundle mean ($\mu$)')
                    plt.xlabel('node index')
                    plt.ylabel(f'{scalar_name} values')
                    plt.legend()
                    plt.show()

                    plt.figure(figsize=(10,2))
                    plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} heatmap')
                    im = plt.imshow(values.T, cmap='hot', interpolation='nearest')
                    plt.xlabel('streamline index')
                    plt.ylabel('node index')
                    add_colorbar(im)
                    plt.show()

                plt.figure()
                plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} mean ($\mu$)-reference warped streamline profiles')
                plt.plot(dtw_values.T, c='tab:blue', alpha=0.1)
                plt.plot(mean_values.T, c='black', label='bundle mean ($\mu$)')
                plt.plot(dtw_mean_values.T, c='black', label='warped bundle mean ($\mu^{\prime}$)', linestyle='dashed')
                plt.xlabel('node index')
                plt.ylabel(f'{scalar_name} values')
                plt.legend()
                f_name = op.join(target_dir, f'streamline_{truncated_name}_warp_profile.png')
                print(f_name)
                plt.savefig(f_name, bbox_inches = "tight")
                plt.show()

                plt.figure()
                plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} mean ($\mu$)-reference warped heatmap')
                im = plt.imshow(dtw_values.T, cmap='hot', interpolation='nearest')
                plt.xlabel('streamline index')
                plt.ylabel('node index')
                add_colorbar(im)
                f_name = op.join(target_dir, f'streamline_{truncated_name}_warp_heatmap.png')
                print(f_name)
                plt.savefig(f_name, bbox_inches = "tight")
                plt.show()

##### Pairwise Warped Streamlines

Warp each streamline to each other creates NxN matrix

In [ ]:
# TODO load from file instead of recalculating
for subject in subjects:
    if compare_test_retest:
        loc_test  = get_iloc(myafq_test, subject)
        loc_retest = get_iloc(myafq_retest, subject)
    else:
        loc = get_iloc(myafq, subject)

    for bundle_name in bundle_names:
        if compare_test_retest:
            iterables = zip(test_retest_names, test_retest_sessions, [myafq_test, myafq_retest], [loc_test, loc_retest])
        else:
            iterables = zip([sub_dir], [dataset_name], [myafq], [loc])
            
        for name, ses, myafq, loc in iterables:
            print(name, subject, bundle_name, ses)
            target_dir = get_dir_name(myafq, sub_dir, bundle_name, loc)
            fgarray = set_number_of_points(streamlines[subject][bundle_name][ses], n_points)
            
            if len(fgarray) == 0:
                print(f'No streamlines for {name} {subject} {bundle_name} {ses}')
                continue

            for scalar_name, data in zip(scalars, scalar_data[subject][ses]):
                truncated_name = scalar_name.split('_')[-1]
                values = np.array(values_from_volume(data, fgarray, affines[subject][bundle_name][ses]))
                
                mean_values = np.mean(values, axis=0)

                tic = time.perf_counter()
                
                dtw_values = np.zeros((values.shape[0], values.shape[0], values.shape[1]))
                
                for i, a in enumerate(values):
                    for j, b in enumerate(values):
                        _, path = fastdtw(a,b)
                        path = np.array(path)
                        dtw_value = a[np.append(path[np.where(path[:,1][:-1] != path[:,1][1:]),0][0], len(values.T)-1)]
                        dtw_values[i,j] = dtw_value
                
                toc = time.perf_counter()
                print(f'dtw calculation {toc - tic:0.4f} seconds')
                print(dtw_values.shape)
                
                if not compare_test_retest:
                    f_name = op.join(target_dir, f'streamline_profile_pairwise_warped_{truncated_name}.npy')
                    print(f_name)
                    np.save(f_name, dtw_values)

                x, y = np.meshgrid(np.arange(dtw_values.shape[0]),np.arange(dtw_values.shape[1]))
                
                show_all_n_points = True
                
                # this show the full volume with all interior points
                # illustrative (to get context for surface plots)
                # it is computationally intensive, and not really necessary
                if show_all_n_points:
                    tic = time.perf_counter()

                    fig = plt.figure()
                    fig.suptitle(f'{name} {subject} {bundle_name} {scalar_name} {ses} pairwise warped streamline profiles wireframes')
                    ax=fig.add_subplot(111,projection='3d')
                    for i in range(dtw_values.shape[2]):
                        ax.plot_wireframe(x, y, dtw_values[:,:,i])
    #                     ax.plot_surface(x, y, dtw_values[:,:,i])
                    ax.set_xlabel('streamline index')
                    ax.set_ylabel('streamline index')
                    ax.set_zlabel(f'{scalar_name} values')
                    f_name = op.join(target_dir, f'streamline_{truncated_name}_pairwise_warp_profile_full.png')
                    print(f_name)
                    plt.savefig(f_name, bbox_inches = "tight")
                    plt.show()
                    toc = time.perf_counter()
                    print(f'plot {toc - tic:0.4f} seconds')
            
                tic = time.perf_counter()
        
                # what really want to do is find external surfaces (min and max)
                # TODO: how to create a wireframe of suface connecting min and max plane
                # TODO: downsample no need to show every combination
                fig = plt.figure()
                fig.suptitle(f'{name} {subject} {bundle_name} {scalar_name} {ses} pairwise warped streamline profiles min-max wireframes')
                ax=fig.add_subplot(111,projection='3d')
                ax.plot_wireframe(x, y, np.amax(dtw_values, axis=2))
                ax.plot_wireframe(x, y, np.amin(dtw_values, axis=2))
                ax.set_xlabel('streamline index')
                ax.set_ylabel('streamline index')
                ax.set_zlabel(f'{scalar_name} values')
                f_name = op.join(target_dir, f'streamline_{truncated_name}_pairwise_warp_profile.png')
                print(f_name)
                plt.savefig(f_name, bbox_inches = "tight")
                plt.show()
                
                toc = time.perf_counter()
                print(f'plot {toc - tic:0.4f} seconds')
                            
                show_exemplar = True
                
                # this should output a profile and heatmap that warp one streamline to another 
                # (as opposed to the mean as done above)
                if show_exemplar:
                    example = np.random.choice(dtw_values.shape[0])
                    
                    print(f'selecting streamline {example} from {name} {subject} {bundle_name} {scalar_name} {ses}')
                    exemplar_dtw_values = dtw_values[example]
                    dtw_mean_values = np.mean(exemplar_dtw_values, axis=0)
                                        
                    if show_warp_map:
                        # show how each streamline warps to the sampled streamline based on tissue property
                        plt.figure()
                        plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} pairwise reference warping map for streamline {example}')
                        for value in values:
                            plt.plot(*zip(*fastdtw(value.T, exemplar_dtw_values[example].T)[1]), c='tab:blue')
                        plt.plot(*zip(*fastdtw(values[example].T, exemplar_dtw_values[example].T)[1]), c='k', label=f'streamline {example}')
                        plt.xlabel('input node index')
                        plt.ylabel('output node index')
                        plt.legend()
                        f_name = op.join(target_dir, f'{truncated_name}_exemplar_pairwise_warp_map.png')
                        print(f_name)
                        plt.savefig(f_name, bbox_inches = "tight")
                        plt.show()

                    # show the streamline with the mean and the warped mean
                    # to get sense of how warping to this streamline effects the meam
                    plt.figure()
                    plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} target pairwise streamline {example}')
                    plt.plot(mean_values.T, c='k', label='bundle mean ($\mu$)')
                    plt.plot(dtw_mean_values.T, c='k', linestyle='dashed', label='exemplar mean ($\mu^{\prime}$)')
                    # these are same as maps directly to itself
#                     plt.plot(exemplar_dtw_values[example].T,  c='tab:blue', label='exemplar dtw values')
                    plt.plot(values[example].T, c='tab:blue', label='exemplar values')
                    plt.xlabel('node index')
                    plt.ylabel(f'{scalar_name} values')
                    plt.legend()
                    f_name = op.join(target_dir, f'streamline_{truncated_name}_exemplar_pairwise_warp.png')
                    print(f_name)
                    plt.savefig(f_name, bbox_inches = "tight")
                    plt.show()
                    
                    # show all the streamlines before warping to sampled streamline
                    plt.figure()
                    plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} streamline profiles {example}')
                    plt.plot(values.T, c='tab:blue', alpha=0.1)
                    plt.plot(mean_values.T, c='k', label='bundle mean ($\mu$)')
                    plt.plot(dtw_mean_values.T, c='k', linestyle='dashed', label='exemplar mean ($\mu^{\prime}$)')
                    plt.xlabel('node index')
                    plt.ylabel(f'{scalar_name} values')
                    plt.legend()
                    f_name = op.join(target_dir, f'streamline_{truncated_name}_exemplar_pairwise_profile.png')
                    print(f_name)
                    plt.savefig(f_name, bbox_inches = "tight")
                    plt.show()

                    # show all the streamlines after warping to sampled streamline
                    plt.figure()
                    plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} pairwise warped streamline profiles for streamline {example}')
                    plt.plot(exemplar_dtw_values.T, c='tab:blue', alpha=0.1)
                    plt.plot(mean_values.T, c='black', label='bundle mean ($\mu$)')
                    plt.plot(dtw_mean_values.T, c='black', label='exemplar mean ($\mu^{\prime}$)', linestyle='dashed')
                    plt.xlabel('node index')
                    plt.ylabel(f'{scalar_name} values')
                    plt.legend()
                    f_name = op.join(target_dir, f'streamline_{truncated_name}_exemplar_pairwise_warp_profile.png')
                    print(f_name)
                    plt.savefig(f_name, bbox_inches = "tight")
                    plt.show()

                    # show the effect of warping on the tissue property values
                    plt.figure()
                    plt.title(f'{name} {subject} {bundle_name} {scalar_name} {ses} pairwise warped heatmap for streamline {example}')
                    im = plt.imshow(exemplar_dtw_values.T, cmap='hot', interpolation='nearest')
                    plt.xlabel('streamline index')
                    plt.ylabel('node index')
                    add_colorbar(im)
                    f_name = op.join(target_dir, f'streamline_{truncated_name}_exemplar_pairwise_warp_heatmap.png')
                    print(f_name)
                    plt.savefig(f_name, bbox_inches = "tight")
                    plt.show()

### <span style="color:blue">**TODO: Other Warps**</span>